Se va a desarroyar un código para etiquetar las imágenes usando los modelos pre entrenados de ultralytics para entrenar previamente las etiquetas y que el etiquetado de las mismas sea más leve

In [5]:
import os
import cv2
from accuracy_functions import ULTRALYTICSMetrics
from ultralytics_models_utils import model

yolo_metrics=ULTRALYTICSMetrics()
rtdetr_model=model('rtdetr-l.pt','rtdetr')


print_=False

images_path_out='/home/ana/TFM/datasets/AUTOLABELING'
images_path_in='/home/ana/TFM/datasets/desperfectos_moviles_reduccion_clases.v1i.coco/valid'

names_path="classes_premodel.txt"
names = rtdetr_model.model.names

# Obtenemos un array con las imágenes de la carpeta
imagenes = os.listdir(images_path_in)
# Recorremos el array de imágenes
for imagen in imagenes:
    # Juntamos la ruta de la carpeta con el nombre de la imagen
    image_path = os.path.join(images_path_in, imagen)
    try:
        labels_detected, _=rtdetr_model.detected_labels_model(image_path,"xywhn")
        #codición para filtrar solo las etiquetas de cell phone
        labels_detected = [label for label in labels_detected if names[label[0]] == 'cell phone']
        image_detected, predicted_boxes=yolo_metrics.get_boxes_image(image_path, labels_detected, names_path,True)
        # Guardamos la imagen detectada
        cv2.imwrite(os.path.join(images_path_out, imagen), image_detected)

        width = image_detected.shape[1]
        height = image_detected.shape[0]      
        rtdetr_model.save_predicted_boxes_to_txt(predicted_boxes, os.path.join(images_path_out,imagen[:-4] + '.txt'), names_path,height, width)
    except Exception as e:
        print(e)
        print("Error en ",image_path)


No images or videos found in /home/ana/TFM/datasets/desperfectos_moviles_reduccion_clases.v1i.coco/valid/_annotations.coco.json. Supported formats are:
images: {'tiff', 'bmp', 'mpo', 'webp', 'pfm', 'png', 'dng', 'jpeg', 'tif', 'jpg'}
videos: {'mov', 'mkv', 'mp4', 'avi', 'webm', 'gif', 'mpeg', 'asf', 'ts', 'wmv', 'm4v', 'mpg'}
Error en  /home/ana/TFM/datasets/desperfectos_moviles_reduccion_clases.v1i.coco/valid/_annotations.coco.json


#### Adaptar las clases de los archivos txt para reducir las etiquetas de coco a mi modelo personalizado de cell phone

In [6]:
import os

# Define la ruta a la carpeta que contiene los archivos .txt
folder_path = '/home/ana/TFM/datasets/AUTOLABELING'

# Itera sobre todos los archivos en la carpeta especificada
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Asegúrate de procesar solo archivos .txt
        file_path = os.path.join(folder_path, filename)  # Construye la ruta completa al archivo
        
        # Leer el contenido del archivo
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # Procesar cada línea para cambiar la clase a 0
        new_lines = []
        for line in lines:
            parts = line.split()  # Divide la línea en partes
            parts[0] = '0'  # Cambia la clase al valor 0
            new_lines.append(" ".join(parts) + '\n')  # Reconstruye la línea y agrega un salto de línea
        
        # Sobrescribe el archivo con las nuevas líneas
        with open(file_path, 'w') as file:
            file.writelines(new_lines)

print("Todos los archivos han sido actualizados.")


Todos los archivos han sido actualizados.


##### Ahora divido el dataset para entrenar el modelo

In [7]:
import os
import shutil
import random

# Define la ruta a la carpeta que contiene los archivos .jpg y .txt
input_folder = '/home/ana/TFM/datasets/AUTOLABELING'

# Rutas para las carpetas de salida
output_base = '/home/ana/TFM/datasets/autolabel_review'
train_folder = os.path.join(output_base, 'train')
valid_folder = os.path.join(output_base, 'valid')
test_folder = os.path.join(output_base, 'test')

# Crea las carpetas necesarias
for folder in [train_folder, valid_folder, test_folder]:
    os.makedirs(os.path.join(folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(folder, 'labels'), exist_ok=True)

# Leer todos los archivos y filtrar por archivos .jpg
files = [f for f in os.listdir(input_folder) if f.endswith('.jpg') and os.path.exists(os.path.join(input_folder, f.replace('.jpg', '.txt')))]

# Mezclar los archivos aleatoriamente
random.shuffle(files)

# Calcular los índices para dividir los datos
num_files = len(files)
train_end = int(num_files * 0.8)
valid_end = train_end + int(num_files * 0.15)

# Dividir los archivos
train_files = files[:train_end]
valid_files = files[train_end:valid_end]
test_files = files[valid_end:]

# Función para copiar archivos a sus nuevas ubicaciones
def copy_files(files, folder):
    for f in files:
        # Copiar imagen
        shutil.copy(os.path.join(input_folder, f), os.path.join(folder, 'images', f))
        # Copiar archivo de etiqueta correspondiente
        txt_file = f.replace('.jpg', '.txt')
        shutil.copy(os.path.join(input_folder, txt_file), os.path.join(folder, 'labels', txt_file))

# Copiar archivos a las respectivas carpetas
copy_files(train_files, train_folder)
copy_files(valid_files, valid_folder)
copy_files(test_files, test_folder)

print("Archivos distribuidos correctamente en train, valid y test.")


Archivos distribuidos correctamente en train, valid y test.
